# <center> ÉCONOMÉTRIE S2 L3 MIASH</center>
## <center> TRAVAIL No 1 : </center>
## <center> RÉGRESSION LINÉAIRE </center>
#### <center>Michal Urdanivia (UGA)</center>
#### <center> michal.wong-urdanivia@univ-grenoble-alpes.fr </center>

In [1]:
import pandas as pd
import statsmodels.api as sm

Dictionnaire des données: http://fmwww.bc.edu/ec-p/data/wooldridge/kielmc.des

In [2]:
url = 'https://www.stata.com/data/jwooldridge/eacsap/hprice.dta'
kieldat = pd.read_stata(url)
kieldat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321 entries, 0 to 320
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      321 non-null    int16  
 1   age       321 non-null    int16  
 2   agesq     321 non-null    float32
 3   nbh       321 non-null    int8   
 4   cbd       321 non-null    float32
 5   intst     321 non-null    float32
 6   lintst    321 non-null    float32
 7   price     321 non-null    float32
 8   rooms     321 non-null    int8   
 9   area      321 non-null    int16  
 10  land      321 non-null    float32
 11  baths     321 non-null    int8   
 12  dist      321 non-null    float32
 13  ldist     321 non-null    float32
 14  lprice    321 non-null    float32
 15  y81       321 non-null    int8   
 16  larea     321 non-null    float32
 17  lland     321 non-null    float32
 18  lintstsq  321 non-null    float32
dtypes: float32(12), int16(3), int8(4)
memory usage: 20.7 KB


In [4]:
print(kieldat.head()) #Affichage des première lignes.
kieldat.describe() # Quelques statistiques descriptives.

   year  age   agesq  nbh     cbd   intst  lintst    price  rooms  area  \
0  1978   48  2304.0    4  3000.0  1000.0  6.9078  60000.0      7  1660   
1  1978   83  6889.0    4  4000.0  1000.0  6.9078  40000.0      6  2612   
2  1978   58  3364.0    4  4000.0  1000.0  6.9078  34000.0      6  1144   

     land  baths     dist     ldist     lprice  y81     larea     lland  \
0  4578.0      1  10700.0  9.277999  11.002100    0  7.414573  8.429017   
1  8370.0      2  11000.0  9.305651  10.596635    0  7.867871  9.032409   
2  5000.0      1  11500.0  9.350102  10.434115    0  7.042286  8.517193   

    lintstsq  
0  47.717705  
1  47.717705  
2  47.717705  


,year,age,agesq,nbh,cbd,intst,lintst,price,rooms,area,land,baths,dist,ldist,lprice,y81,larea,lland,lintstsq
count,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000
mean,1979.327103,18.009346,1381.567017,2.208723,15822.429688,16442.367188,9.480508,96100.656250,6.585670,2106.728972,39629.890625,2.339564,20715.576172,9.837417,11.378119,0.442368,7.597237,10.301861,90.482254
std,1.492329,32.565845,4801.790039,2.164353,8967.102539,9033.133789,0.777166,43223.722656,0.901204,694.957902,39514.378906,0.770526,8508.184570,0.478383,0.438175,0.497443,0.340723,0.801752,14.066459
min,1978.000000,0.000000,0.000000,0.000000,1000.000000,1000.000000,6.907800,26000.000000,4.000000,735.000000,1710.000000,1.000000,5000.000000,8.517193,10.165852,0.000000,6.599871,7.444249,47.717705
25%,1978.000000,0.000000,0.000000,0.000000,9000.000000,9000.000000,9.105000,65000.000000,6.000000,1560.000000,16935.000000,2.000000,13400.000000,9.503010,11.082143,0.000000,7.352441,9.737138,82.901016
50%,1978.000000,4.000000,16.000000,2.000000,14000.000000,16000.000000,9.680300,85900.000000,7.000000,2056.000000,43560.000000,2.000000,19900.000000,9.898475,11.360939,0.000000,7.628518,10.681894,93.708206
75%,1981.000000,22.000000,484.000000,4.000000,23000.000000,24000.000000,10.086000,120000.000000,7.000000,2544.000000,46100.000000,3.000000,27200.000000,10.210970,11.695247,1.000000,7.841493,10.738568,101.727402
max,1981.000000,189.000000,35721.000000,6.000000,35000.000000,34000.000000,10.434000,300000.000000,10.000000,5136.000000,544500.000000,4.000000,40000.000000,10.596630,12.611538,1.000000,8.544030,13.207623,108.868355


On estime l'équation MCO:

\begin{align*}
\log(price_i) =& \alpha + \log(dist_i)\beta + U_i
\end{align*}


où $price_i$ est le prix d'une maison $i$, et $dist_i$ sa distance par rapport à un incinérateur d'ordures.

In [5]:
kieldat["const"] = 1
## On utilise la fonction OLS dans Statsmodels.
linreg = sm.OLS(endog = kieldat['lprice'], exog = kieldat[['const', 'ldist']], missing = 'drop')
## Ci-dessus le modèle a été construit. Pour l'estimer on utilise ".fit()" ci-après:
results = linreg.fit()
## Pour avoir un résumé des résultats on peut utiliser ".summary()" comme ici:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     43.48
Date:                Tue, 14 Mar 2023   Prob (F-statistic):           1.78e-10
Time:                        11:03:21   Log-Likelihood:                -169.60
No. Observations:                 321   AIC:                             343.2
Df Residuals:                     319   BIC:                             350.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.2575      0.474     17.427      0.0

Les valeurs prédites sont données par:

\begin{align*}
\hat{\log(price_i)} =& \underbrace{8.2575}_{\hat{\alpha}}  + \log(dist_i)\underbrace{0.3172}_{\hat{\beta}}
\end{align*}



In [9]:

## On utilise la fonction OLS dans Statsmodels.
linreg2 = sm.OLS(endog = kieldat['lprice'], exog = kieldat[['const', 'ldist', 'rooms', 'baths']], missing = 'drop')
## Ci-dessus le modèle a été construit. Pour l'estimer on utilise ".fit()" ci-après:
results2 = linreg2.fit(cov_type="HC0")
## Pour avoir un résumé des résultats on peut utiliser ".summary()" comme ici:
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     115.6
Date:                Tue, 14 Mar 2023   Prob (F-statistic):           1.39e-50
Time:                        11:36:19   Log-Likelihood:                -87.191
No. Observations:                 321   AIC:                             182.4
Df Residuals:                     317   BIC:                             197.5
Df Model:                           3                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.4291      0.358     26.335      0.0